# Критерии остановки и оценка длительности

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

np.random.seed(7)

В предыдущих примерах эксперименты останавливались при достижении 95% вероятности целевой метрики одной группы больше других. Порог 95% произволен - возможны 80%, 99%, другие значения. Удобен объективный критерий остановки.

Качественные закономерности следующие. Продолжение эксперимента дает новые данные и уточняет метрики. При этом часть пользователей попадает в неоптимальную группу, что ведет к потерям. Поддержка эксперимента также связана с потерями из-за усложнения разработки, хотя этот эффект тяжело оценить. Ценность новых данных убывает по мере их накопления - первые 100 точек сильнее влияют на решение, чем дополнительные 100 точек к 10 млн. Наконец, решение об остановке должно учитывать последствия выбора. Чем выше цена ошибки, тем большая нужна уверенность. Например, для лекарств нужно убедиться в отсутствии побочных эффектов, поэтому порог вероятности при тестировании должен быть высоким. В веб-сервисах выбранных вариант может продержатся год до следующего эксперимента.  
  

Таким образом, цена поддержания эксперимента постоянна, ценность новых данных падает. Эксперимент пора останавливать, когда ценность новых данных становится ниже их стоимости. Ценность данных должна учитывать последствия выбора.

<center>
<img src="../figs/ab_data_value.png" alt="ab_data_value" width="600"/>
    
<em>Цена поддержания эксперимента постоянна, ценность новых данных падает. Эксперимент пора останавливать при ценности новых данных ниже их стоимости. Ценность данных должна учитывать последствия выбора одного из вариантов. </em>
</center>

Реализация этих закономерностей зависит от конкретных метрик. Далее будут рассматриваться конверсии. Конверсии можно моделировать последовательностью случайных величин с двумя возможными исходами - успехом и неудачей. Для оценки вероятности успеха $p$ в серии $N$ испытаний с $n_s$ успехами правдоподобие удобно задавать биномиальным распределением $P(\mathcal{D} | \mathcal{H}) = \mbox{Binom}(n_s, N | p)$, априорное распределение - бета-распределением $P(\mathcal{H}) = \mbox{Beta}(p; \alpha, \beta)$ с параметрами $\alpha, \beta$. Тогда апостериорная вероятность также будет бета-распределением с обновленными параметрами $P(\mathcal{H} | \mathcal{D}) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)$. При достаточно большом количестве данных $N \gg n_s \gg \alpha, \beta$ бета-распределение близко нормальному $\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) \approx \mbox{Norm}(x; \mu, \sigma^2), \, \mu = n_s / N, \, \sigma^2 = \mu (1 - \mu) / N$.

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p)
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta)
$$

$$
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
$$


$$
N \gg n_s \gg \alpha, \beta: \quad
\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s) 
\approx \mbox{Norm}(x; \mu, \sigma^2),
\quad
\mu = n_s / N, 
\,
\sigma^2 = \mu (1 - \mu) / N
$$


$$
\mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\qquad
\mbox{Beta}(x; \alpha, \beta) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\qquad
\mbox{Norm}(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
$$

Апостериорные распределения для двух групп показаны на графике ниже. Конверсия в одной группе $p_A = 10\%$, в другой на 5% больше $p_B = 10.5\%$. Из этих групп делается выборка 10000 точек. Апостериорные распределения построены по первой тысяче точек и по всем 10000. Видно, что распределения по 10000 точек уже. Это показывает общую тенденцию - по мере набора данных оценки конверсий уточняются, средние апостериорных распределений приближаются к точным средним, а распределения сужаются.

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

pa = 0.1
pb = pa * 1.05

npoints = [1000, 9000]
sa = stats.binom.rvs(p=pa, n=npoints)
sb = stats.binom.rvs(p=pb, n=npoints)
npoints = np.cumsum(npoints)
sa = np.cumsum(sa)
sb = np.cumsum(sb)

xaxis_min = 0.05
xaxis_max = 0.17
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sa[-1], npoints[-1]).pdf(x), 
                         line_color='red', opacity=0.8, name=f'А, N={npoints[-1]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sa[0], npoints[0]).pdf(x), 
                         line_color='red', opacity=0.3, name=f'А, N={npoints[0]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sb[-1], npoints[-1]).pdf(x), 
                         line_color='blue', opacity=0.8, name=f'B, N={npoints[-1]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sb[0], npoints[0]).pdf(x), 
                         line_color='blue', opacity=0.3, name=f'B, N={npoints[0]}'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[xaxis_min, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

Для оценки эффекта и выбора лучшей группы нужно распределение разности конверсий $P_{p_B - p_A}$. Бета-распределения близки нормальным. Разность случайных величин с нормальным распределением также случайная величина с нормальным распределением. Среднее равно разности средних, диспресия - сумме дисперсий. Поэтому разность приближенно можно считать нормальным распределением $P_{p_B - p_A}(x) \approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right)$. Вероятность конверсии группы Б больше А $P(p_B > p_A)$ равна вероятности разности $p_B - p_A$ больше 0 $P(p_B - p_A > 0)$. Можно посчитать с помощью функции распределения $1 - F_{p_B - p_A}(0)$.

$$
\begin{gather}
P_{p_A} = \mbox{Beta}(x; \alpha + n_{s_A}, \beta + N_A - n_{s_A})
\approx \mbox{Norm}(x; \mu_A, \sigma^2_A),
\\
\\
P_{p_B} = \mbox{Beta}(x; \alpha + n_{s_B}, \beta + N_B - n_{s_B})
\approx \mbox{Norm}(x; \mu_B, \sigma^2_B)
\\
\\
P_{p_B - p_A}(x) = 
\int_{-\infty}^{\infty} dy P_{p_B}(y) P_{p_A}(y-x)
\approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right)
\\
\\
P(p_B > p_A) = P(p_B - p_A > 0) = 1 - F_{p_B - p_A}(0)
\end{gather}
$$

По мере набора данных средние в выборках будут приближаться к точным средним, а апостериорные распределения сужаются. То же для разности. Задано две группы, конверсия в одной $p_A = 10\%$, в другой на 5% больше $p_B = 10.5\%$. В этих группах набираются данные с шагом 1000 точек. На первом графике показаны апостериорные распределения при 1000 и 10000 точек в каждой группе. Видно, что распределения при большем количестве точек уже. На втором графике показаны средние и 95% области наибольшей плотности вероятности по мере набора N. По мере набора данных средние в выборках приближаются к точным средним, а 95% области наибольшей плотности вероятности сужаются. На третьем графике показа вероятность $P(p_B > p_A)$. Она растет по мере накопления данных.

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)
df[['diff_lower', 'diff_upper']] = df.apply(lambda r: (r['diff_mu'] - 2*r['diff_s'], r['diff_mu'] + 2*r['diff_s']), axis=1, result_type="expand")

xaxis_min = -0.1
xaxis_max = 0.1  
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
for N, o in [(1000, 0.3), (10000, 1)]:
    dist_a = posterior_dist_binom(df[df['N'] == N]['sa'], N)
    dist_b = posterior_dist_binom(df[df['N'] == N]['sb'], N)
    #fig.add_trace(go.Scatter(x=x, y=dist_a.pdf(x), line_color='red', opacity=o, name=f'А, N={N}'))
    #fig.add_trace(go.Scatter(x=x, y=dist_b.pdf(x), line_color='blue', opacity=o, name=f'Б, N={N}'))
    #
    diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
    fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', opacity=o, name=f'Diff, N={N}'))
    fig.update_layout(title='$p_B - p_A$',
                      xaxis_title='$p$',
                      yaxis_title='Плотность вероятности',
                      xaxis_range=[xaxis_min, xaxis_max],
                      hovermode="x",
                      height=500)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='$p_A$',
                         line_color='red'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
                         fill="toself", name='$p_A, \, \mbox{95% PDI}$', marker_color='red', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='$p_B$',
                         line_color='blue'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
                         fill="toself", name='$p_B, \, \mbox{95% PDI}$', marker_color='blue', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='$p_B-p_A$',
                         line_color='black'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
                         fill="toself", name='$p_B - p_A, \, \mbox{95% PDI}$', marker_color='black', opacity=0.2))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N',
                  height=600)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
                         line_color='black', name='$p_B > p_A$'))
fig.update_layout(title='$P(p_B > p_A)$',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  yaxis_range=[0, 1],
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)  
fig.show()

Дисперсия бета-распределения убывает пропорционально $1/N$. Это видно как из точного выражения для дисперсии бета-распределения, так и из приближения нормальным распределением. Т.к. дисперсия разности равна сумме дисперсий $p_A, p_B$, для нее эта зависимость так же справедлива. 

$$
\mbox{Var}(\mbox{Beta}(x; \alpha, \beta)) = \frac{\alpha \beta}{(\alpha + \beta)^2(\alpha + \beta + 1)}
\\
\\
N \gg n_s \gg \alpha, \beta:
\quad
\mbox{Var}(\mbox{Beta}(x; \alpha + n_s, \beta + N - n_s)) \approx \frac{n_s (N - n_s)}{N^3} 
= \frac{\mu (1 - \mu)}{N},
\quad
\mu = n_s / N
\\
\mbox{Var}(P_{p_A}), \mbox{Var}(P_{p_B}), \mbox{Var}(P_{{p_B}-{p_A}}) \sim 1/N
$$

Для графика ниже заново сгенерированы 150 тыс. точек в каждой группе с шагом 1000 точек. Показано стандартное отклонение разности по мере набора точек и величина $\sigma_0 \sqrt{N_0/N}$, где $\sigma_0$ и $N_0$ - стандартное отклонение и количество точек на первом шаге набора данных. Величины близки.  

In [ ]:
pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)
df['diff_var'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2, axis=1)


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s'],
                         line_color='black', name='$\sigma_{p_B - p_A}$'))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s'][0] * np.sqrt(df['N'][0] / df['N']), 
                         line_color='black', mode='lines', line_dash='dash', name='sigma0 * sqrt(N0/N)'))
# fig.add_trace(go.Scatter(x=df['N'], y=df['diff_var'],
#                          line_color='black', name='Var'))
# fig.add_trace(go.Scatter(x=df['N'], y=df['diff_var'][0] * df['N'][0] / df['N'], 
#                          line_color='black', mode='lines', line_dash='dash', name='sigma0 * sqrt(N0/N)'))
fig.update_layout(title='Стандартное отклонение pB - pA',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)  
fig.show()

Таким образом, по мере набора данных средние в выборках стремятся к точному среднему, разность средних стремится к точной разности. Дисперсии распределений и разности сужаются пропорционально $1/N$. Растет уверенность в лучшей группе.

Эксперимент имеет смысл останавливать при максимальном ожидаемом выигрыше. Ожидаемый выигрыш при выобре одной из групп пропорционален ожидаемому значению разности целевой метрики в группах. Хотя по мере набора данных разность сходится к точному значению, сказать как именно она будет меняться по имеющимся данным нельзя. Если в каждой группе уже собрано $N$ точек, при добавлении еще $\Delta N$ ожидаемое значение можно считать близким к текущей оценке $E[p_B-p_A | N + \Delta N] \approx E[p_B -p_A | N]$. Если ожидаемая разность сильно не меняется, ожидаемый выигрыш тоже не меняется. Т.к. дисперсия будет убывать, распределение станет немного уже при том же среднем. При одинаковых средних у более узкого распределения ниже возможные потери, чем у более широкого, что предпочтительнее. 

На графике два распределения разности с одинаковыми средними. Ожидаемые потери - среднее по части распределения левее нуля. У более узкого распределения ожидаемые потери меньше, чем у широкого при том же среднем. 

In [ ]:
pa = 0.1
pb = pa * 1.05

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
mu = pb - pa
N = 5000
s = np.sqrt((pb*(1-pb)/N) + (pa*(1-pa)/N))
d = stats.norm(loc=mu, scale=s)
fig.add_trace(go.Scatter(x=x, y=d.pdf(x), 
                         line_color='black', name=f'N={N}'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(d.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.add_trace(go.Scatter(x=x[x<0], y=d.pdf(x[x<0]), fill='tozeroy',
                         line_color='black', opacity=0.3, name='loss', showlegend=False))
N = 1000
s = np.sqrt((pb*(1-pb)/N) + (pa*(1-pa)/N))
d = stats.norm(loc=mu, scale=s)
fig.add_trace(go.Scatter(x=x, y=d.pdf(x), 
                         line_color='black', opacity=0.3, name=f'N={N}'))
fig.add_trace(go.Scatter(x=x[x<0], y=d.pdf(x[x<0]), fill='tozeroy',
                         line_color='black', opacity=0.3, name='loss'))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.05, 0.05],
                  hovermode="x",
                  height=500)
fig.show()

Эксперимент пора останавливать, если разность ожидаемых потерь на следующем шаге $E[L(p_B - p_A | N + \Delta N)]$ и ожидаемых потерь при немедленной остановке $E[L(p_B - p_A | N)]$ меньше стоимости данных $\mbox{Cost}(\Delta N)$. Ожидаемые потери должны учитывать количество будущих пользователей $M$, на которых повлияют изменения, и LTV. Стоимость данных - ожидаемые потери из-за попадания части пользователей в худшую группу. Они определяются как потери конверсии на LTV на долю пользователей в худшей группе $w_A$ на количество пользователей $\Delta N$. 

LTV -> $LTV_{B-A}$ или $\Delta LTV$ - разница LTV в группах 

$$
E[L(p_B - p_A | N + \Delta N)] - E[L(p_B - p_A | N)] < \mbox{Cost}(\Delta N)
\\
\begin{split}
E[L(p_B - p_A | N)] & = LTV \cdot M \int_{-\infty}^0 x P_{p_B - p_A}(x) dx
\\
& = LTV \cdot M \int_{-\infty}^0 x Norm(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2 | N) dx
\\
& = LTV \cdot M \cdot I(N)
\end{split}
$$

$$
\mbox{Cost}(\Delta N) \approx LTV \cdot w_A \Delta N \cdot E[P_{p_B - p_A}]  = LTV \cdot w_A \Delta N \cdot (\mu_B - \mu_A)
$$

$$
LTV \cdot M (I(N+\Delta N) - I(N)) < LTV \cdot w_A (\mu_B - \mu_A) \Delta N
$$

$$
\frac{I(N+\Delta N) - I(N)}{\Delta N} < \frac{w_A}{M} (\mu_B - \mu_A)
$$

Разность можно приближенно заменить производной. Ожидаемые потери аналитически 

$$
\frac{d I(N)}{dN} < \frac{w_A}{M} (\mu_B - \mu_A)
$$

$$
\begin{split}
I(N) &= \int_{-\infty}^{0} x \mathcal N(x; m, \sigma^2) dx
  = \int_{-\infty}^{0} \big(x + m - m\big) \mathcal N(x;m,\sigma^2) dx 
  \\
  & = m \int_{-\infty}^{0} \mathcal N(x;m,\sigma^2) dx + \int_{-\infty}^{0} (x-m) \mathcal N(x;m,\sigma^2) dx 
  \\
  &= m \Phi \left(\tfrac{0-m}{\sigma}\right) + \sigma \int_{-\infty}^{(0-m)/\sigma} t \varphi(t) dt 
  = m \Phi \left(-\tfrac{m}{\sigma}\right) - \sigma \varphi \left(-\tfrac{m}{\sigma}\right)
  \\
  & = m \Phi \left(-\tfrac{m}{\sigma}\right) - \sigma \varphi \left(\tfrac{m}{\sigma}\right)
\end{split}
$$

При $\sigma(N) = \sigma_0 \sqrt{N_0 / N}$:

$$
\begin{split}
I(N) &= m\,\Phi\!\left(-\tfrac{m}{\sigma(N)}\right) \;-\; \sigma(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right), \\[6pt]
\frac{dI}{dN}
&= m\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)\cdot\frac{m\,\sigma'(N)}{\sigma(N)^2}
\;-\;\Big[\sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
+ \sigma(N)\,\varphi'\!\left(\tfrac{m}{\sigma(N)}\right)\cdot\frac{dz}{dN}\Big], \\[6pt]
&= \frac{m^2\sigma'(N)}{\sigma(N)^2}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
- \sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
- \frac{m^2\sigma'(N)}{\sigma(N)^2}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right), \\[6pt]
&= -\,\sigma'(N)\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right).
\end{split}
\\
\sigma'(N) = -\sigma(N) / 2 N 
\\
\frac{dI}{dN} \;=\; \frac{\sigma(N)}{2N}\,\varphi\!\left(\tfrac{m}{\sigma(N)}\right)
= \frac{\sigma(N)^2}{2N} Norm\left(0; \mu_B - \mu_A, \sigma^2 \right)
$$

Условие остановки
$$
\frac{\sigma(N)^2}{2N} Norm\left(0; \mu_B - \mu_A, \sigma^2 \right) < \frac{w_A}{M} (\mu_B - \mu_A)
\\
Norm\left(0; \mu_B - \mu_A, \sigma^2 \right) < 2 w_A \frac{N}{M} \frac{(\mu_B - \mu_A)}{\sigma(N)^2}
\\
\phi(z) < 2 w_A \frac{N}{M} z, \quad z = \frac{\mu_B - \mu_A}{\sigma}, \quad \phi(z) = \frac{1}{\sqrt{2\pi}} e^{-x^2/2}
$$

Ограничение хвостов нормального распределения https://en.wikipedia.org/wiki/Mills_ratio .  
Т.к. $u/z \ge 1, u \in [z, \infty)$

$$
\Phi(-z) 
= \frac{1}{\sqrt{2\pi}}\int_{z}^{\infty} e^{-u^{2}/2}\,du
\;\le\; \frac{1}{\sqrt{2\pi}}\int_{z}^{\infty} \frac{u}{z} e^{-u^{2}/2}\,du
= \frac{1}{z\sqrt{2\pi}} e^{-z^{2}/2}
= \frac{\varphi(z)}{z}, \qquad z>0 .
$$

Условие остановки: вероятность группа B хуже (левый хвост) ограничена сверху $Norm(x)/x$ меньше $2w_A N/M$. 

$$
F(0; \mu_B - \mu_A, \sigma^2) < \frac{\phi(z)}{z} < 2w_A \frac{N}{M}
$$

Если бы средние по мере набора данных не менялись, графики выглядели бы как в примере ниже. Разные $M$ соответствуют разным вероятностям остановки.

In [ ]:
pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

mu = pb - pa
df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['N'] = df['npoints'].cumsum()
df['s'] = np.sqrt((pb*(1-pb)/df['N']) + (pa*(1-pa)/df['N']))
df['pbgtpa'] = df['s'].apply(lambda s: 1 - stats.norm(loc=mu, scale=s).cdf(0))
df['stopping_lhs'] = df['s'].apply(lambda s: s**2 / mu * stats.norm.pdf(0, loc=mu, scale=s))

wA = 0.5
M1 = 100000
M2 = 1000000
M3 = 10000000
df['stopping_rhs_m1'] = (2 * wA / M1) * df['N'] 
df['stopping_rhs_m2'] = (2 * wA / M2) * df['N'] 
df['stopping_rhs_m3'] = (2 * wA / M3) * df['N'] 

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_lhs'], 
                         line_color='black', mode='lines', name='LHS'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m1'], 
                         line_color='black', line_dash='dash', opacity=0.3, mode='lines', name='RHS, M=1e5'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m2'], 
                         line_color='black', line_dash='longdash', opacity=0.3, mode='lines', name='RHS, M=1e6'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m3'], 
                         line_color='black', line_dash='longdashdot', opacity=0.3, mode='lines', name='RHS, M=1e7'))
fig.add_trace(go.Scatter(x=df['N'], y=df['pbgtpa'],
                         line_color='black', opacity=0.3, name='$P(p_B - p_A > 0)$'))
fig.update_layout(title='Критерий остановки',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  yaxis_range=[0, 1],
                  height=500)  
fig.show()

print('Stopping condition LHS < RHS:')
for M, mi in [(M1,'m1'), (M2,'m2'), (M3,'m3')]:
    N, pbgtpa = df.iloc[np.argmax(df['stopping_lhs'] < df[f'stopping_rhs_{mi}'])][['N', 'pbgtpa']]
    print(f'M={M}: N={int(N)}, pb>pa: {pbgtpa*100:.2f}%')

Таким образом, для предполагаемого эффекта и выбранного количества пользоватлей, на которых повлияет раскатка, можно оценить пороговую вероятность остановки эксперимента.

Среднее меняется по мере набора данных.  
Поэтому графики выглядят менее плавно.

In [ ]:
pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)


wA = 0.5
M1 = 100000
M2 = 1000000
M3 = 10000000

df['stopping_lhs'] = df.apply(lambda r: r['diff_s']**2 / r['diff_mu'] * stats.norm.pdf(0, loc=r['diff_mu'], scale=r['diff_s']), axis=1)
df['stopping_rhs_m1'] = (2 * wA / M1) * df['N'] 
df['stopping_rhs_m2'] = (2 * wA / M2) * df['N'] 
df['stopping_rhs_m3'] = (2 * wA / M3) * df['N'] 

#todo: add numerical lhs

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
                         line_color='black', name='$p_B > p_A$'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_lhs'], mode='lines', name='LHS'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m1'], mode='lines', name='RHS, M=1e5'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m2'], mode='lines', name='RHS, M=1e6'))
fig.add_trace(go.Scatter(x=df['N'], y=df['stopping_rhs_m3'], mode='lines', name='RHS, M=1e7'))
fig.update_layout(title='Condition vs N (RHS linear in N)',
                  xaxis_title='$N$',
                  yaxis_title='Value',
                  hovermode="x unified",
                  yaxis_range=[0, 1],
                  height=500)
fig.show()


# df_flt = df[5:]
# display(df_flt.iloc[[(df_flt['stopping_lhs'] < df_flt['stopping_rhs_m1']).idxmax()]][['N', 'pb_gt_pa']])
# display(df_flt.iloc[[(df_flt['stopping_lhs'] < df_flt['stopping_rhs_m2']).idxmax()]][['N', 'pb_gt_pa']])
# display(df_flt.iloc[[(df_flt['stopping_lhs'] < df_flt['stopping_rhs_m3']).idxmax()]][['N', 'pb_gt_pa']])

Оценка длительности до фиксированной вероятности $P(p_B - p_A) > 0$.   
Нужно задать базовое значени $p_A$.   
Предположить эффект $dp$ или базовое значение $p_B$.  
Ожидаемый эффект $p_B - p_A$.  
Для конверсий дисперсия связана со средним $\sigma^2 = p(1-p)/N$.  
Дисперсия разности - сумма дисперсий групп $\sigma_A^2 + \sigma_B^2$.  
Можно посчитать, как меняется $P(p_B - p_A > 0)$ по мере роста $N$: $P(p_B > p_A) = 1 - F_{p_B - p_A}(0)$  
Найти минимальное $N$, для которого вероятность $P(p_B - p_A > 0)$ больше пороговой.  

$N$ для групп по отдельности. 
Чаще удобнее суммировать. 


Для средних дисперсия не связана со средним.   
Нужна оценка дисперсии по историческим данным. 

$$
\begin{gather}
P_{p_A} = \mbox{Beta}(x; \alpha + n_{s_A}, \beta + N_A - n_{s_A})
\approx \mbox{Norm}(x; \mu_A, \sigma^2_A),
\\
\\
P_{p_B} = \mbox{Beta}(x; \alpha + n_{s_B}, \beta + N_B - n_{s_B})
\approx \mbox{Norm}(x; \mu_B, \sigma^2_B)
\\
\\
P_{p_B - p_A}(x) = 
\int_{-\infty}^{\infty} dy P_{p_B}(y) P_{p_A}(y-x)
\approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right)
\\
\\
P(p_B > p_A) = P(p_B - p_A > 0) = 1 - F_{p_B - p_A}(0)
\end{gather}
$$



In [ ]:
pa = 0.1
pb = pa * 1.05

dp = pb - pa  
N = np.arange(100, 100000, 100)
s = np.sqrt(pa * (1 - pa) / N + pb * (1 - pb) / N)
pbgtpa = np.array([1 - stats.norm(loc=dp, scale=s).cdf(0) for s in s])
p_stop = 0.95

fig = go.Figure()
fig.add_trace(go.Scatter(x=N, y=pbgtpa,
                         line_color='black', line_dash='solid', name='pb_gt_pa, fixed mean'))
fig.add_hline(p_stop, line_dash='dash')
fig.add_vline(N[np.argmax(pbgtpa > p_stop)], line_dash='dash')
fig.update_layout(title='P(pB > pA)',
                  xaxis_title='$N$',
                  yaxis_range=[0, 1.05],
                  hovermode="x",
                  height=500)  
fig.show()

N[np.argmax(pbgtpa > p_stop)]

Сравнить оценку с моделированием.  
Распределение длительности при фиксированном эффекте.  

In [ ]:
pa = 0.1
pb = pa * 1.05

p_stop = 0.93

dp = pb - pa  
N = np.arange(100, 100000, 100)
s = np.sqrt(pa * (1 - pa) / N + pb * (1 - pb) / N)
pbgtpa = np.array([1 - stats.norm(loc=dp, scale=s).cdf(0) for s in s])
Nest = N[np.argmax(pbgtpa > p_stop)]

Nstop = []
won = []
Nexp = 10000
nmax = 5000000
nstep = 1000
for i in range(Nexp):
    if i % 1000 == 0: print(f'Nexp: {i}')
    N = 0
    sa = 0
    sb = 0
    while N < nmax:
        N += nstep
        sa += stats.binom.rvs(p=pa, n=nstep)
        sb += stats.binom.rvs(p=pb, n=nstep)
        mua = sa / N
        mub = sb / N
        s = np.sqrt(mua * (1 - mua) / N + mub * (1 - mub) / N)
        pbgtpa = 1 - stats.norm(loc=mub-mua, scale=s).cdf(0)
        if pbgtpa > p_stop or pbgtpa < 1 - p_stop:
            bestgr = 'B' if pbgtpa > p_stop else 'A'
            won.append(bestgr)
            Nstop.append(N)
            #print(f'Exp {i}: Nstop {N}')
            break

#print(Nest, Nstop)
            
freq = pd.DataFrame({
    'total': pd.Series(won).value_counts().sort_index(),
})
freq['frequency'] = freq['total'] / freq['total'].sum()
display(freq)

fig = go.Figure()
fig.add_trace(go.Histogram(x=Nstop, nbinsx=100))
fig.add_vline(Nest)
fig.show()

print(f'Est.N: {Nest}')
print(f'Exp ended before estimate: {np.sum(Nstop < Nest), np.sum(Nstop < Nest) / Nexp}')

Распределение - обратное биномиальное?  
Количество попыток до определенного числа успехов p_stop?

Сумма двух обратных биномиальных (p>p_stop + p<p_stop)?

Как записать условие $P(p_B > p_A) = 1 - F_{p_B - p_A}(0)$ в терминах отдельных точек?

Можно ли думать о наборе данных как о случайном процессе?  
https://en.wikipedia.org/wiki/Stochastic_process  
https://en.wikipedia.org/wiki/Random_walk  
https://en.wikipedia.org/wiki/Wiener_process  

https://en.wikipedia.org/wiki/First-hitting-time_model  
https://en.wikipedia.org/wiki/Hitting_time  

https://en.wikipedia.org/wiki/Gambler%27s_ruin  
https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE_%D1%80%D0%B0%D0%B7%D0%BE%D1%80%D0%B5%D0%BD%D0%B8%D0%B8_%D0%B8%D0%B3%D1%80%D0%BE%D0%BA%D0%B0

Еще вариант: на каждом шаге случайная величина
{pb>pa: +1, pb=pa: 0, pb<pa:-1}

На практике эффект неизвестен заранее.  

Еще одна проблема - много ошибок при ранних остановках.  



Много ошибок из-за ранних остановок.   
Чем меньше nstep, тем больше ошибок.  
Чем больше nstep, тем ближе доля ошибок к заявленной.  

Попробовать выставлять минимальное время для эксп.  
Но основании ожидаемого эффекта.  
Скажем, треть времени.  

Оценка минимальной длительности:  
изменение дисперсии много меньше величины эффекта?  
$$
\delta \sigma \ll \Delta \mu
$$
Или нет.
Меняется оценка среднего, дисперсия сильно не меняется.  

Насколько сильно изменится оценка если добавить еще точек.

Предположим, текущая оценка соответствует неверному решению.  
Причем, если уверенность в неверной оценке большая - это должно быть редкое событие.  
Как она поменяется, если добавить еще точек?  
Считать длительность для изменения решения.

Минимальный эффект: 
$$
2 \sigma < \Delta \mu
$$

Считать время на изменение решения.  
Готовы ли держать еще столько времени?  
Это способ оценить расходы на поддержание эксперимента на глаз.

In [ ]:
fig = go.Figure()
fig.add_histogram(x=nstop, xbins={'start':0, 'size':nstep, 'end':nmax})
fig.add_vline(Ndr)
fig.show()

vals, counts = np.unique(won, return_counts=True)
print('Best gr: ', dict(zip(vals, counts)))

Разброс непредсказуемый.  
Даже если угадать эффект, сделанная оценка длительности говорит не ясно о чем. 

Вместо точной оценки может хватить оценки сверху.  

В реальности эффект заранее неизвестен.   
Корректировать оценку при появлении данных.  
- Предположить эффект.  
- Оценить длительность.

Что будет, если использовать другой критерий остановки?  
Какая будет точность?  

Если между группами небольшая разница, не интересно дожидаться вероятности 95%.  
Займет много времени, но смысла в этом нет.  

Если эффект меньше заданной величины, эксперимент пора прекращать.  
Отсюда же можно строить оценки длительности.

Как записать?

$$
\frac{p_B - p_A}{p_A} < 0.03
$$

Относительная разность меньше 3% с вероятностью 95% .
$$
P\left(\frac{p_B - p_A}{p_A} < 0.03\right) > 95 \%
$$

$$
P\left(\frac{p_B}{p_A} < 1.03\right) > 95 \%
$$

Нужно учесть знак.

$$
\frac{|p_B - p_A|}{p_A} < 0.03
$$

При нормальных распределениях pb, pa по распространению ошибок

Если распределения $p_A$ и $p_B$ нормальные, то распределение $\Delta p$ также можно приближенно считать нормальным. 
Также нужно $\mathrm{Cov}(p_A, p_B) = 0$ и малая плотность вероятности $p_A$ вблизи 0.

$$
P_{p_A}(x) = \text{Norm}(x; \mu_A, \sigma^2_A),
\qquad
P_{p_B}(x) = \text{Norm}(x; \mu_B, \sigma^2_B)
\\
P_{\Delta p}(x) \approx \text{Norm}(x; \mu_{\Delta}, \sigma_{\Delta}^2), 
\quad
\mu_{\Delta} = \frac{\mu_B - \mu_A}{\mu_A},
\quad
\sigma_{\Delta} = \frac{|\mu_B|}{|\mu_A|}
\sqrt{
\frac{\sigma_{A}^{2}}{\mu_A^{2}}
+ \frac{\sigma_{B}^{2}}{\mu_B^{2}}
}
$$

Для отношения:
$$
P_{p_A}(x) = \text{Norm}(x; \mu_A, \sigma^2_A),
\qquad
P_{p_B}(x) = \text{Norm}(x; \mu_B, \sigma^2_B)
\\
P_{B/A}(x) \approx \text{Norm}(x; \mu_{B/A}, \sigma_{B/A}^2), 
\quad
\mu_{B/A} = \frac{\mu_B}{\mu_A},
\quad
\sigma_{B/A} = \frac{|\mu_B|}{|\mu_A|}
\sqrt{
\frac{\sigma_{A}^{2}}{\mu_A^{2}}
+ \frac{\sigma_{B}^{2}}{\mu_B^{2}}
}
$$

Для оценок конверсий:
$$
\mu = p, \quad \sigma^2 = p(1-p)/N
$$

In [ ]:
pa = 0.1
pb = 0.1 * 1.03

rel_thr = 1.01
p_stop = 0.99

N = np.arange(100, 1_000_000, 1000)
spa = np.sqrt(pa * (1 - pa) / N)
spb = np.sqrt(pb * (1 - pb) / N)
rel_d = pb / pa
rel_s = pb / pa * np.sqrt(spa**2 / pa**2 + spb**2 / pb**2)
#rel_dist = stats.norm(loc=rel_d, scale=rel_s)
#p_thr = rel_dist.cdf(rel_thr)
p_thr = [stats.norm.cdf(rel_thr, loc=rel_d, scale=s) for s in rel_s]

fig = go.Figure()
fig.add_trace(go.Scatter(x=N, y=p_thr,
                         line_color='black', line_dash='solid', name='pb_gt_pa, fixed mean'))
fig.add_hline(p_stop, line_dash='dash')
fig.add_hline(1-p_stop, line_dash='dash')
#fig.add_vline(N[np.argmax(pbgtpa > p_stop)], line_dash='dash')
fig.update_layout(title=f'P(pB/pA < {rel_thr})',
                  xaxis_title='$N$',
                  yaxis_range=[0, 1.05],
                  hovermode="x",
                  height=500)  
fig.show()


Для относительной разности граница постоянная.  
Можно применить задачи касания стенки.  

Ссылки

https://en.wikipedia.org/wiki/Expected_value_of_sample_information  

https://en.wikipedia.org/wiki/Truncated_normal_distribution  

https://en.wikipedia.org/wiki/Mills_ratio

https://en.wikipedia.org/wiki/Conjugate_prior

https://en.wikipedia.org/wiki/Posterior_predictive_distribution

https://en.wikipedia.org/wiki/Beta-binomial_distribution

Ожидаемое значение след. шага равно последнему известному значению.  
https://en.wikipedia.org/wiki/Martingale_(probability_theory)

Начать с абсолютной разности.

$$
P(|p_B - p_A| < c)
\\
P_{p_B - p_A}(x) 
\approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right)
\\
\\
\begin{split}
P(|p_B - p_A| < c) & = P(p_B - p_A < c) + P(p_B - p_A > 1 - c) 
\\
& = F_{p_B - p_A}(c) + 1 - F_{p_B - p_A}(1 - c)
\\
& = 1 - 2 F_{p_B - p_A}(c)
\end{split}
$$

Условие остановки: либо одна группа лучше другой $P(B>A) > 95\%$, либо разница несущественна $P(|B-A| < c) > 95\%$.

То же для относительной разности.

$$
P(\frac{|p_B - p_A|}{p_A} < c)
$$

Следующий вопрос - как выбрать $c$ и $95\%$?  
Здесь понадобится оценка последствий.  

Вероятность одной группы лучше другой связана с оценкой эффекта?

Нарисовать картинку с динамикой.  
Разность по мере набора данных.  
Полоса [-c, c].

In [ ]:
c = 2.5
x= np.linspace(-7, 7, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x),
                        line_color='black'))
fig.add_trace(go.Scatter(x=[c, c], y=[0, max(stats.norm.pdf(x)*1.1)],
                         mode='lines', line_dash='dash', line_color='black'))
fig.add_trace(go.Scatter(x=[-c, -c], y=[0, max(stats.norm.pdf(x)*1.1)],
                         mode='lines', line_dash='dash', line_color='black'))
fig.update_layout(
    height=500,
    template='plotly_white')
fig.show()

Оценки длительности

Есть оценка распределения. Или априорное распределение. Как оценить длительность эксп?

При остановке по достижении вероятности:



In [ ]:
mu = pb - pa
s = np.sqrt((pb*(1-pb)/df['N']) + (pa*(1-pa)/df['N']))
pbgtpa = [1-stats.norm(loc=mu, scale=s).cdf(0) for s in s]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
                         line_color='red', name='pb_gt_pa, simulated'))
fig.add_trace(go.Scatter(x=df['N'], y=pbgtpa,
                         line_color='black', line_dash='dash', name='pb_gt_pa, fixed mean'))
fig.update_layout(title='P(pB > pA)',
                  xaxis_title='$N$',
                  yaxis_range=[0, 1.05],
                  hovermode="x",
                  height=500)  
fig.show()

Проверки

Фиксированный эффект.  
Оценивается длительность.  
Сравнивается с фактической в неск. эксп.  

In [ ]:
mu = pb - pa
N = np.arange(1000, 150000, 1000)
s = [np.sqrt((pb*(1-pb)/n) + (pa*(1-pa)/n)) for n in N]
pbgtpa = np.array([1-stats.norm(loc=mu, scale=s).cdf(0) for s in s])

Ndr = N[pbgtpa > 0.95][0]

fig = go.Figure()
fig.add_trace(go.Scatter(x=N, y=pbgtpa,
                         line_color='black', line_dash='dash', name='pb_gt_pa, fixed mean'))
fig.add_vline(Ndr)
fig.update_layout(title='P(pB > pA)',
                  xaxis_title='$N$',
                  yaxis_range=[0, 1.05],
                  hovermode="x",
                  height=500)  
fig.show()

In [ ]:
pa = 0.1
pb = pa * 1.03
p_stop = 0.95
nmax = 5000000
nstep = 100

nexp = 1000

won = []
nstop = []
for ne in range(nexp):
    N = np.arange(nstep, nmax + nstep, nstep)
    sa = stats.binom.rvs(p=pa, n=nstep, size=nmax//nstep)
    sb = stats.binom.rvs(p=pb, n=nstep, size=nmax//nstep)
    Sa = sa.cumsum()
    Sb = sb.cumsum()
    Pa = Sa / N
    Pb = Sb / N
    #for n, pa_n, pb_n in zip(N, Pa, Pb):
        #mu = pb_n - pa_n
        #sigma = np.sqrt((pb_n*(1-pb_n)/n) + (pa_n*(1-pa_n)/n))
    for n, sa_n, sb_n in zip(N, Sa, Sb):
        post_a = posterior_dist_binom(sa_n, n)
        post_b = posterior_dist_binom(sb_n, n)
        mu = post_b.mean() - post_a.mean()
        sigma = np.sqrt(post_a.std()**2 + post_b.std()**2)
        pbgtpa = 1 - stats.norm(loc=mu, scale=sigma).cdf(0)
        pbest = np.maximum(pbgtpa, 1-pbgtpa)
        if pbest > p_stop:
            #nstop.append(np.argmax(pbest > p_stop))
            nstop.append(n)
            #won.append('B' if pb_n > pa_n else 'A')
            won.append('B' if sb_n > sa_n else 'A')
            break
#nstop

fig = go.Figure()
fig.add_histogram(x=nstop, xbins={'start':0, 'size':nstep, 'end':nmax})
fig.add_vline(Ndr)
fig.show()

vals, counts = np.unique(won, return_counts=True)
print('Best gr: ', dict(zip(vals, counts)))

$$
\begin{split}
P_{p_A}(x) = \mbox{Beta}(x; n_{s_A} + \alpha, N_A - n_{s_A} + \beta)
& \approx \mbox{Norm}(x; \mu_A, \sigma_A^2),
\quad
\mu_A = (n_{s_A} + \alpha - 1) /N_A, 
\,
\sigma_A^2 = \mu_A (1 - \mu_A) / N_A,
\quad
N_A \gg n_{s_A} \gg 1
\\
\\
P_{p_B}(x) = \mbox{Beta}(x; n_{s_B} + \alpha, N_B - n_{s_B} + \beta)
& \approx \mbox{Norm}(x; \mu_B, \sigma_B^2),
\quad
\mu_B = (n_{s_B} + \alpha - 1)/N_B, 
\,
\sigma_B^2 = \mu_B (1 - \mu_B) / N_B,
\quad
N_B \gg n_{s_B} \gg 1
\\
\\
P_{p_B - p_A}(x) = 
\int_{-\infty}^{\infty} dy P_{p_B}(y) P_{p_A}(y-x)
& \approx \mbox{Norm}\left(x; \mu_B - \mu_A, \sigma_A^2 + \sigma_B^2\right),
\quad
\mbox{Norm}(x ; \mu, \sigma^2) \equiv \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
\end{split}
$$

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)
df['diff_mu'] = df.apply(lambda r: r['pb'] - r['pa'], axis=1)
df['diff_s'] = df.apply(lambda r: np.sqrt(posterior_dist_binom(r['sa'], r['N']).std()**2 + posterior_dist_binom(r['sb'], r['N']).std()**2), axis=1)
df[['diff_lower', 'diff_upper']] = df.apply(lambda r: (r['diff_mu'] - 2*r['diff_s'], r['diff_mu'] + 2*r['diff_s']), axis=1, result_type="expand")
#todo: loss

xaxis_min = -0.05
xaxis_max = 0.15  
#x = np.linspace(0, xaxis_max, 1000)
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
for N, o in [(1000, 0.3), (10000, 1)]:
    dist_a = posterior_dist_binom(df[df['N'] == N]['sa'], N)
    dist_b = posterior_dist_binom(df[df['N'] == N]['sb'], N)
    fig.add_trace(go.Scatter(x=x, y=dist_a.pdf(x), line_color='red', opacity=o, name=f'А, N={N}'))
    fig.add_trace(go.Scatter(x=x, y=dist_b.pdf(x), line_color='blue', opacity=o, name=f'Б, N={N}'))
    #
    diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
    fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', opacity=o, name=f'Diff, N={N}'))
    fig.update_layout(title='Апостериорные распределения',
                      xaxis_title='$p$',
                      yaxis_title='Плотность вероятности',
                      xaxis_range=[xaxis_min, xaxis_max],
                      hovermode="x",
                      height=500)
#     fig.update_layout(title='Апостериорные распределения',
#                       xaxis_title='$p$',
#                       yaxis_title='Плотность вероятности',
#                       xaxis_range=[0, xaxis_max],
#                       hovermode="x",
#                       height=500)
fig.show()


# xaxis_min = -0.05
# xaxis_max = 0.15    
# x = np.linspace(xaxis_min, xaxis_max, 1000)
# fig = go.Figure()
# for N in [1000, 10000]:
#     diff = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])
#     fig.add_trace(go.Scatter(x=x, y=diff.pdf(x), line_color='black', name=f'Diff, N={N}'))
#     fig.update_layout(title='Апостериорные распределения',
#                       xaxis_title='$p$',
#                       yaxis_title='Плотность вероятности',
#                       xaxis_range=[xaxis_min, xaxis_max],
#                       hovermode="x",
#                       height=500)
# fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='A',
                         line_color='red'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
                         fill="toself", name='A, 95% PDI', marker_color='red', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='B',
                         line_color='blue'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
                         fill="toself", name='B, 95% PDI', marker_color='blue', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='Diff',
                         line_color='black'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
                         fill="toself", name='A, 95% PDI', marker_color='black', opacity=0.2))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N',
                  height=700)
fig.show()


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['diff_mu'], name='$p_B - p_A$',
#                          line_color='black'))
# fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
#                          y=list(df['diff_lower']) + list(reversed(df['diff_upper'])),
#                          fill="toself", name='$p_B - p_A, \mbox{ 95% PDI}$', marker_color='black', opacity=0.2))
# fig.update_layout(title='$p_B - p_A$',
#                   yaxis_tickformat = ',.1%',
#                   xaxis_title='N')
# fig.show()


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'], name='P(pb > pa)',
#                          line_color='black'))
# fig.update_layout(title='$P(p_B > p_A)$',
#                   yaxis_range=[0, 1],
#                   xaxis_title='N')
# fig.show()

In [ ]:
N = 5000
approx_diff_dist = stats.norm(loc=df[df['N'] == N]['diff_mu'], scale=df[df['N'] == N]['diff_s'])

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='$\mbox{Аналитическое приближение}$'))
# fig.add_trace(go.Scatter(x=[approx_diff_dist.mean()[0], approx_diff_dist.mean()[0]], 
#                          y=[0, max(approx_diff_dist.pdf(x))], 
#                          line_color='black', mode='lines', name='mean'))
fig.add_trace(go.Scatter(x=x[x<0], y=approx_diff_dist.pdf(x[x<0]), fill='tozeroy',
                         line_color='black', opacity=0.3, name='loss'))
# fig.add_trace(go.Scatter(x=x[x>=0], y=approx_diff_dist.pdf(x[x>=0]), 
#                          line_color='black', fillcolor='white', name='gain', fill='tozeroy'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()

In [ ]:
mu = pb - pa
s = np.sqrt((pb*(1-pb)/df['N']) + (pa*(1-pa)/df['N']))
pbgtpa = [1-stats.norm(loc=mu, scale=s).cdf(0) for s in s]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['diff_s']*10,
                         line_color='red', name='s*10, simulated'))
fig.add_trace(go.Scatter(x=df['N'], y=s*10,
                         line_color='black', line_dash='dash', name='s*10, fixed mean'))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'],
                         line_color='red', name='pb_gt_pa, simulated'))
fig.add_trace(go.Scatter(x=df['N'], y=pbgtpa,
                         line_color='black', line_dash='dash', name='pb_gt_pa, fixed mean'))
#todo: loss
fig.update_layout(title='Стандартное отклонение pB - pA',
                  xaxis_title='$N$',
                  #yaxis_title='sigma',
                  #xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)  
fig.show()

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

pa = 0.1
pb = pa * 1.05

npoints = [1000, 9000]
sa = stats.binom.rvs(p=pa, n=npoints)
sb = stats.binom.rvs(p=pb, n=npoints)
npoints = np.cumsum(npoints)
sa = np.cumsum(sa)
sb = np.cumsum(sb)

xaxis_min = 0.05
xaxis_max = 0.17
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sa[-1], npoints[-1]).pdf(x), 
                         line_color='red', opacity=0.8, name=f'А, N={npoints[-1]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sa[0], npoints[0]).pdf(x), 
                         line_color='red', opacity=0.3, name=f'А, N={npoints[0]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sb[-1], npoints[-1]).pdf(x), 
                         line_color='blue', opacity=0.8, name=f'B, N={npoints[-1]}'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(sb[0], npoints[0]).pdf(x), 
                         line_color='blue', opacity=0.3, name=f'B, N={npoints[0]}'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[xaxis_min, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

pa = 0.1
pb = pa * 1.05

npoints = 1000
nstep = 150
sa = stats.binom.rvs(p=pa, n=npoints, size=nstep)
sb = stats.binom.rvs(p=pb, n=npoints, size=nstep)

df = pd.DataFrame()
df['npoints'] = [npoints] * nstep
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['npoints'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")


xaxis_min = 0.05
xaxis_max = 0.17
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(df[df['N'] == 10000]['sa'], 10000).pdf(x), 
                         line_color='red', opacity=0.8, name=f'А, N=10000'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(df[df['N'] == 1000]['sa'], 1000).pdf(x), 
                         line_color='red', opacity=0.3, name=f'А, N=1000'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(df[df['N'] == 10000]['sb'], 10000).pdf(x), 
                         line_color='blue', opacity=0.8, name=f'B, N=10000'))
fig.add_trace(go.Scatter(x=x, y=posterior_dist_binom(df[df['N'] == 1000]['sb'], 1000).pdf(x), 
                         line_color='blue', opacity=0.3, name=f'B, N=1000'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[xaxis_min, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='$p_A$',
                         line_color='red'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
                         fill="toself", name='$p_A, \, \mbox{95% PDI}$', marker_color='red', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='$p_B$',
                         line_color='blue'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
                         fill="toself", name='$p_B, \, \mbox{95% PDI}$', marker_color='blue', opacity=0.2))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N',
                  height=500)
fig.show()

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

a, b = -np.inf, (0 - approx_diff_dist.mean()) / approx_diff_dist.std()
tr_loss = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='Diff'))
fig.add_trace(go.Scatter(x=x, y=tr_loss.pdf(x), 
                         line_color='red', name='Loss'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


dN = 100

res = []
for N in range(na+nb, 100_000, dN):
    mu = approx_diff_dist.mean()
    s = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)
    new_diff_dist = stats.norm(loc=approx_diff_dist.mean(), scale=s)
    p = 1 - new_diff_dist.cdf(0)
    a, b = -np.inf, (0 - new_diff_dist.mean()) / new_diff_dist.std()
    tr_loss = stats.truncnorm(a=a, b=b, loc=new_diff_dist.mean(), scale=new_diff_dist.std())
    mean_loss = tr_loss.mean() * new_diff_dist.cdf(0)
    s2 = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N+dN)
    new_diff_dist2 = stats.norm(loc=approx_diff_dist.mean(), scale=s2)
    a2, b2 = -np.inf, (0 - new_diff_dist2.mean()) / new_diff_dist2.std()
    tr_loss2 = stats.truncnorm(a=a2, b=b2, loc=new_diff_dist2.mean(), scale=new_diff_dist2.std())
    mean_loss2 = tr_loss2.mean() * new_diff_dist2.cdf(0)
    d_loss = (mean_loss2 - mean_loss) / dN
    loss_diff = new_diff_dist.pdf(0) * s**2 / (2 * N)
    #loss_diff = stats.norm.pdf(mu/s) * s / (2 * N)
    res.append((N, mu, s, p, mean_loss, d_loss, loss_diff))
    
#res = [(N, approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)) for N in range(na+nb, 1000000, 100)]

df_res = pd.DataFrame(res, columns=['N', 'mu', 'std', 'p', 'mean_loss', 'd_loss', 'loss_diff'])
df_res


fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['N'], y=df['std'] * 100, 
#                         line_color='blue', name='std * 100'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['p'], 
                         line_color='black', name='p'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['mean_loss'] * 100, 
                         line_color='red', name='mean_loss * 100'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['mu']*10, 
                         line_color='yellow', name='mu*10'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['d_loss'] * 1000000, 
                         line_color='green', name='d_loss * 1000000'))
fig.add_trace(go.Scatter(x=df_res['N'], y=df_res['loss_diff'] * 1000000, 
                         line_color='purple', name='loss_diff * 1000000'))
fig.show()